# 102 Spark basics

The goal of this lab is to get familiar with Spark programming.

- [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
- [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)

In [ ]:
import org.apache.spark

Intitializing Scala interpreter ...

In [ ]:
// DO NOT EXECUTE - this is needed just to avoid showing errors in the following cells
val sc = spark.SparkContext.getOrCreate()

## 102-1 Spark warm-up

Load the ```capra``` and ```divinacommedia``` datasets and try the following actions:
- Show their content (```collect```)
- Count their rows (```count```)
- Split phrases into words (```map``` or ```flatMap```; what’s the difference?)
- Check the results (remember: evaluation is lazy)
- Try the ```toDebugString``` function to check the execution plan

In [2]:
val rddCapra = sc.textFile("../../../../datasets/capra.txt")
val rddDC = sc.textFile("../../../../datasets/divinacommedia.txt")

rddCapra: org.apache.spark.rdd.RDD[String] = ../../../../datasets/capra.txt MapPartitionsRDD[1] at textFile at <console>:25
rddDC: org.apache.spark.rdd.RDD[String] = ../../../../datasets/divinacommedia.txt MapPartitionsRDD[3] at textFile at <console>:26


In [ ]:
val rddCapraWords1 = rddCapra.map( x => x.split(" ") )
rddCapraWords1.collect()

In [ ]:
rddCapraWords1.count()

In [ ]:
val rddCapraWords2 = rddCapra.flatMap( x => x.split(" ") )
rddCapraWords2.collect()

In [ ]:
rddCapraWords2.count()

In [ ]:
val rddL = rddCapra.
   flatMap( x => x.split(" ") ).
   map(x => (x,1)).
   reduceByKey((x,y)=>x+y)
rddL.toDebugString

## 102-2 Basic Spark jobs

Implement on Spark the following jobs and test them on both capra and divinacommedia datasets.

- **Word count**: count the number of occurrences of each word
  - Result: (sopra, 1), (la, 4), …
- **Word length count**: count the number of occurrences of words of given lengths
  - Result: (2, 4), (5, 8)
- Count the average length of words given their first letter (i.e., words that begin with "s" have an average length of 5)
  - Result: (s, 5), (l, 2), …
- Return the inverted index of words (i.e., for each word, list the numbers of lines in which they appear)
  - Result: (sopra, (0)), (la, (0, 1)), ...

Also, check how sorting works and try to sort key-value RDDs by descending values.

In [ ]:
// Word count

rddCapra.
  flatMap( x => x.split(" ") ).
  map( x => (x,1)).
  reduceByKey((x,y) => x + y).
  collect()

In [ ]:
// Word length count

rddCapra.
  flatMap( x => x.split(" ") ).
  map( x => (x.length,1)).
  reduceByKey((x,y) => x + y).
  collect()

In [ ]:
// Average word length by initial

rddCapra.
  flatMap( x => x.split(" ") ).
  filter( _.length>0 ).
  map( x => (x.substring(0,1).toLowerCase, (1.0,x.length.toDouble))).
  reduceByKey((x, y) => (x._1 + y._1, x._2 + y._2)).
  mapValues(v => v._2/v._1).
  collect()

In [ ]:
// Inverted index (word-based offset)

rddCapra.
  flatMap( x => x.split(" ") ).
  zipWithIndex().
  groupByKey().
  collect()

In [ ]:
// Inverted index (sentence-based offset)
rddCapra.
  zipWithIndex().
  flatMap({case (k,v)=> k.split(" ").map(x=>(x,v))}).
  distinct().
  groupByKey().
  collect()

In [3]:
// Inverted index (sentence-based offset) alternative
val rddMap = rddCapra.zipWithIndex().
    map({case (k,v)=>(v,k)}).
    flatMapValues( x => x.split(" ") ).
    map({case (k,v)=>(v,k)}).
    distinct().
    groupByKey().
    collect()

rddMap: Array[(String, Iterable[Long])] = Array((campa,CompactBuffer(0)), (la,CompactBuffer(0, 1)), (panca,CompactBuffer(1, 0)), (sotto,CompactBuffer(1)), (crepa,CompactBuffer(1)), (sopra,CompactBuffer(0)), (capra,CompactBuffer(0, 1)))


In [ ]:
// Word count sorted by key

rddCapra.
  flatMap( x => x.split(" ") ).
  map( x => (x,1)).
  reduceByKey((x,y) => x + y).
  sortByKey().
  collect()

In [ ]:
// Word count sorted by descending values

rddCapra.
  flatMap( x => x.split(" ") ).
  map( x => (x,1)).
  reduceByKey((x,y) => x + y).
  map({case (k,v)=>(v,k)}).
  sortByKey(false).
  map({case (k,v)=>(v,k)}).
  collect()

## 103-3 Extra Spark jobs

Implement the following job.

- Co-occurrence count: count the number of co-occurrences in the text. A co-occurrence is defined as "two distinct words appearing in the same line".
  - In the first line of the *capra* dataset, co-occurrences are:
     - (sopra, la), (sopra, panca), (sopra, capra), (sopra, campa)
     - (la, sopra), (la, panca), (la, capra), (la, campa) 
     - (panca, sopra), (panca, la), (panca, capra), (panca, campa)
     - (capra, sopra), (capra, la), (capra, panca), (capra, campa)
     - (campa, sopra), (campa, la), (campa, panca), (campa, capra)